# Final Assignment for Data Basics

### Creating database hospital

In [1]:
def connect():
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="localhost",
                                  port="5432",
                                  )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    return connection

In [2]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
try:
    connection = connect()
    query = "CREATE DATABASE hospital;"
    
    
    cursor = connection.cursor()
    
    # Executing a SQL query
    cursor.execute(query)
    print("Database sucessfully created")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")

Database sucessfully created

PostgreSQL connection is closed


### Defining connect() method so we can we it laster through out our code

In [3]:
#Defining connect() method so we can we it laster through out our code to connect to the database
def connect():
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="localhost",
                                  port="5432",
                                  database="hospital")
    
    return connection

## Creating tables in the database hospital

In [4]:
import logging
try:
    connection = connect()
    
    cursor = connection.cursor()
    
    query_1 = '''
    CREATE TABLE physician (
    employeeid INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    position TEXT NOT NULL,
    ssn INTEGER 
    )
    '''
    cursor.execute(query_1)
    
    query_2 = '''  
    CREATE TABLE department (
    departmentid INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    head INTEGER,
    CONSTRAINT fk_department_physician 
    FOREIGN KEY(head) REFERENCES physician(employeeid)
    
    )
    '''
    cursor.execute(query_2)
    
    query_3 = '''  
    CREATE TABLE affiliated_with (
    physician INTEGER ,
    department INTEGER,
    primaryaffiliation BOOLEAN NOT NULL,
    
    CONSTRAINT fk_affiliated_with_physician
    FOREIGN KEY(physician) REFERENCES physician(employeeid),
    
    CONSTRAINT fk_affiliated_with_department 
    FOREIGN KEY(department) REFERENCES department(departmentid),
    
    PRIMARY KEY (physician , department)
    
    )
    '''
    
    cursor.execute(query_3)
    
    query_4 = '''  
    CREATE TABLE procedure (
    code INTEGER PRIMARY KEY,
    name TEXT,
    cost REAL
    
    )
    '''
    cursor.execute(query_4)
    
    query_5 = '''  
    CREATE TABLE trained_in (
    physician INTEGER ,
    treatment INTEGER,
    certificationdate DATE,
    certificationexpires DATE,
    
    CONSTRAINT fk_trained_in_physician
    FOREIGN KEY(physician) REFERENCES physician(employeeid),
    
    CONSTRAINT fk_trained_in_procedure 
    FOREIGN KEY(treatment) REFERENCES procedure(code),
    
    PRIMARY KEY (physician,treatment)
    
    )
    '''
    
    cursor.execute(query_5)
    
    query_6 =''' 
    CREATE TABLE patient (
    ssn INTEGER PRIMARY KEY,
    name TEXT,
    address TEXT,
    phone TEXT,
    insuranceid INT UNIQUE,
    pcp INTEGER NOT NULL,
    
    CONSTRAINT fk_patient_physician 
    FOREIGN KEY(pcp) REFERENCES physician(employeeid)
    
    )
    '''
    
    cursor.execute(query_6)
    
    query_7 = '''
    CREATE TABLE nurse (
    employeeid INTEGER PRIMARY KEY,
    name TEXT,
    position TEXT,
    registered BOOLEAN NOT NULL,
    ssn INTEGER
    )
    '''
    
    cursor.execute(query_7)
    
    query_8 = '''
    CREATE TABLE appointment (
    appointmentid INTEGER PRIMARY KEY,
    patient INTEGER,
    prepnurse INTEGER,
    physician INT,
    start_dt_time TIMESTAMP NOT NULL,
    end_dt_time TIMESTAMP,
    examinationroom TEXT,
    
    CONSTRAINT fk_appointment_patient 
    FOREIGN KEY(patient) REFERENCES patient(ssn),
    
    CONSTRAINT fk_appointment_nurse 
    FOREIGN KEY(prepnurse) REFERENCES nurse(employeeid),
    
    CONSTRAINT fk_appointment_physician 
    FOREIGN KEY(physician) REFERENCES physician(employeeid)
    )
    
    '''
    
    cursor.execute(query_8)
    
    query_9 = '''
    CREATE TABLE medication (
    code INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    brand TEXT NOT NULL,
    description TEXT
    
    )
    '''
    
    cursor.execute(query_9)
    
    query_10 = '''
    CREATE TABLE prescribes (
    physician INTEGER,
    patient INTEGER,
    medication INTEGER,
    date TIMESTAMP,
    appointment INTEGER,
    dose TEXT,
    
    CONSTRAINT fk_prescribes_physician
    FOREIGN KEY(physician) REFERENCES physician(employeeid),
    
    CONSTRAINT fk_prescribes_patient
    FOREIGN KEY(patient) REFERENCES patient(ssn),
    
    CONSTRAINT fk_prescribes_medication 
    FOREIGN KEY(medication) REFERENCES medication(code),
    
    CONSTRAINT fk_prescribes_appointment
    FOREIGN KEY(appointment) REFERENCES appointment(appointmentid),
    
    PRIMARY KEY(physician,patient,medication,date)
    
    
    )
    '''
    
    cursor.execute(query_10)
    
    query_11 = '''
    CREATE TABLE block (
    blockfloor INTEGER ,
    blockcode INTEGER,
    PRIMARY KEY(blockfloor,blockcode)
    )
    '''
    
    cursor.execute(query_11)
    
    
    query_12 = '''
    CREATE TABLE room (
    roomnumber INTEGER PRIMARY KEY,
    roomtype TEXT,
    blockfloor INTEGER,
    blockcode INTEGER,
    unavailable BOOLEAN NOT NULL,
    
    CONSTRAINT fk_room_block 
    FOREIGN KEY(blockfloor,blockcode) REFERENCES block(blockfloor,blockcode)
    
    )
    '''
    
    cursor.execute(query_12)
    
    query_13 = '''
    CREATE TABLE on_call (
    nurse INTEGER,
    blockfloor INTEGER,
    blockcode INTEGER,
    oncallstart TIMESTAMP,
    oncallend TIMESTAMP,
    
    CONSTRAINT fk_on_call_nurse 
    FOREIGN KEY(nurse) REFERENCES nurse(employeeid),
    
    CONSTRAINT dk_on_call_block
    FOREIGN KEY(blockfloor,blockcode) REFERENCES block(blockfloor,blockcode),
    
    PRIMARY KEY(nurse,blockfloor,blockcode,oncallstart,oncallend)
    
    )
    '''
    
    cursor.execute(query_13)
    
    query_14 = '''
    CREATE TABLE stay (
    stayid INTEGER PRIMARY KEY,
    patient INTEGER,
    room INTEGER,
    start_time TIMESTAMP,
    end_time TIMESTAMP,
    
    CONSTRAINT fk_stay_patient
    FOREIGN KEY(patient) REFERENCES patient(ssn),
    
    CONSTRAINT fk_stay_room 
    FOREIGN KEY(room) REFERENCES room(roomnumber)
    
    )
    '''
    
    cursor.execute(query_14)
    
    query_15 = '''
    CREATE TABLE undergoes (
    patient INTEGER,
    procedure INTEGER,
    stay INTEGER,
    date TIMESTAMP,
    physician INTEGER,
    assistingnurse INTEGER,
    
    CONSTRAINT fk_undergoes_patient 
    FOREIGN KEY(patient) REFERENCES patient(ssn),
    
    CONSTRAINT fk_undergoes_procedure 
    FOREIGN KEY(procedure) REFERENCES procedure(code),
    
    CONSTRAINT fk_undergoes_stay 
    FOREIGN KEY(stay) REFERENCES stay(stayid),
    
    CONSTRAINT fk_undergoes_physician 
    FOREIGN KEY(physician) REFERENCES physician(employeeid),
    
    PRIMARY KEY(patient,procedure,stay,date)
    )
    '''
    
    cursor.execute(query_15)
    
    connection.commit()
    
    
    
    print("Tables sucessfully created")
except (Exception,psycopg2.Error) as error:
    #print("Error while connecting to PostgresSQL: ",error)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)
finally:
    if(connection):
        cursor.close()
        print('')
        print("PostgreSQL connection is closed")
        

Tables sucessfully created

PostgreSQL connection is closed


## Populating data to hospital database from .xlsx file.

In [5]:
import pandas as pd
import psycopg2
import logging

In [6]:
tables=['physician','department','affiliated_with','procedure','trained_in','patient','nurse','appointment','medication','prescribes','block','room','on_call','stay','undergoes']

In [7]:
connection=connect()
cursor = connection.cursor()

try:
    for table in tables:
        df = pd.read_excel ('Data of Final Assignment for Data Basics.xlsx',sheet_name=table)
        for columns in df.columns:
            #print(columns)
            try:
                if df[columns].dtypes =='float':
                    print(str(columns))
                    df[columns]=df[columns].astype('int32')
            except:
                print(str(columns))
                pass

        data = [list(x) for x in df.itertuples(index=False)]
        query = "INSERT INTO " + str(table)+" VALUES(" +'%s'+',%s'*(len(df.columns)-1) + ');'

        print(query)
        for item in data:
            to_push=item
            for i,ele in enumerate(item):
                if str(ele)=='nan':
                    to_push[i]=None
                    
            cursor.execute(query,to_push)
            connection.commit()
        print("Data sucessufully inserted of table "+str(table))
        
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        print('')
        print("PostgreSQL connection is closed")

employeeid
ssn
INSERT INTO physician VALUES(%s,%s,%s,%s);
Data sucessufully inserted of table physician
departmentid
head
INSERT INTO department VALUES(%s,%s,%s);
Data sucessufully inserted of table department
physician
department
INSERT INTO affiliated_with VALUES(%s,%s,%s);
Data sucessufully inserted of table affiliated_with
 code
cost
INSERT INTO procedure VALUES(%s,%s,%s);
Data sucessufully inserted of table procedure
physician
treatment
INSERT INTO trained_in VALUES(%s,%s,%s,%s);
Data sucessufully inserted of table trained_in
ssn
insuranceid
pcp
INSERT INTO patient VALUES(%s,%s,%s,%s,%s,%s);
Data sucessufully inserted of table patient
employeeid
ssn
INSERT INTO nurse VALUES(%s,%s,%s,%s,%s);
Data sucessufully inserted of table nurse
appointmentid
patient
prepnurse
prepnurse
physician
INSERT INTO appointment VALUES(%s,%s,%s,%s,%s,%s,%s);
Data sucessufully inserted of table appointment
code
description
description
INSERT INTO medication VALUES(%s,%s,%s,%s);
Data sucessufully inserted

# Questions

### 1.Write a query in SQL to obtain the name of the physicians who are the head of each department.

In [8]:
import psycopg2
import pandas as pd

In [9]:
conn =connect()
cur = conn.cursor()

query = '''SELECT * FROM physician
INNER JOIN department
    ON physician.employeeid = department.head

'''
    
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,employeeid,name,position,ssn,departmentid,name,head
0,4,Percival Cox,Senior Attending Physician,444444444,1,General Medicine,4
1,7,John Wen,Surgical Attending Physician,777777777,2,Surgery,7
2,9,Molly Clock,Attending Psychiatrist,999999999,3,Psychiatry,9


### 2.Write a query in SQL to count the number of patients who booked an appointment with at least one physician.


In [10]:
cur = conn.cursor()

query = '''SELECT DISTINCT(name) FROM patient
INNER JOIN appointment
    ON patient.ssn = appointment.patient

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,name
0,Dennis Doe
1,Grace Ritchie
2,John Smith
3,Random J. Patient


### 3.Write a query in SQL to obtain the name of the physician and the departments they are affiliated with.


In [11]:
cur = conn.cursor()

query = '''SELECT p.name ,d.name AS department_name
FROM physician AS p
INNER JOIN affiliated_with AS a
    ON p.employeeid = a.physician
INNER JOIN department AS d
    ON d.departmentid = a.department

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,name,department_name
0,John Dorian,General Medicine
1,Elliot Reid,General Medicine
2,Christopher Turk,General Medicine
3,Christopher Turk,Surgery
4,Percival Cox,General Medicine
5,Bob Kelso,General Medicine
6,Todd Quinlan,Surgery
7,John Wen,General Medicine
8,John Wen,Surgery
9,Keith Dudemeister,General Medicine


### 4.Write a query in SQL to count the number of available rooms.


In [12]:
cur = conn.cursor()

query = '''SELECT COUNT(*)  FROM room
WHERE unavailable =False

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,count
0,29


##### Cross cheking if the count of False in unavailable column is really 29

In [13]:
cur = conn.cursor()

query = '''SELECT * FROM room

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())

In [14]:
len(df['unavailable'])-sum(df['unavailable'])

29

### 5.Write a query in SQL to obtain the name of the physicians who are trained for a special treatment.

In [15]:
cur = conn.cursor()

query = '''SELECT p.name as physician_name,procedure.name as procedure_name  FROM physician AS p
INNER JOIN trained_in AS t
    ON p.employeeid = t.physician
INNER JOIN procedure
    ON t.treatment = procedure.code

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,physician_name,procedure_name
0,Christopher Turk,Reverse Rhinopodoplasty
1,Christopher Turk,Obtuse Pyloric Recombobulation
2,Christopher Turk,Obfuscated Dermogastrotomy
3,Christopher Turk,Reversible Pancreomyoplasty
4,Christopher Turk,Follicular Demiectomy
5,Todd Quinlan,Obtuse Pyloric Recombobulation
6,Todd Quinlan,Obfuscated Dermogastrotomy
7,Todd Quinlan,Reversible Pancreomyoplasty
8,John Wen,Reverse Rhinopodoplasty
9,John Wen,Obtuse Pyloric Recombobulation


### 6.Write a query in SQL to obtain the name of the physicians with departments who are yet to be affiliated. 


In [16]:
query = '''
SELECT p.name , d.name
FROM physician AS p
INNER JOIN affiliated_with AS a
    ON p.employeeid = a.physician
INNER JOIN department AS d
    ON a.department = d.departmentid
WHERE a.primaryaffiliation = 'false'

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,name,name
0,Christopher Turk,General Medicine
1,John Wen,General Medicine


### 7.Write a query in SQL to obtain the name of the physicians who are not a specialized physician.

In [17]:
query = '''
SELECT p.name AS "Physician", p.position as "Position"
FROM physician p
LEFT JOIN trained_in t ON p.employeeid=t.physician
WHERE t.treatment IS NULL
ORDER BY employeeid;


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,Physician,Position
0,John Dorian,Staff Internist
1,Elliot Reid,Attending Physician
2,Percival Cox,Senior Attending Physician
3,Bob Kelso,Head Chief of Medicine
4,Keith Dudemeister,MD Resident
5,Molly Clock,Attending Psychiatrist


### 8.Write a query in SQL to find the name of the patients and the number of physicians they have taken appointments.


In [18]:
query = '''
SELECT p.name AS "Patient Name", physician.name as "Physician Name" ,COUNT(*)
FROM patient AS p
INNER JOIN appointment as a
    ON p.ssn = a.patient
INNER JOIN physician 
    ON a.physician =physician.employeeid
GROUP BY p.name,physician.name 


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,Patient Name,Physician Name,count
0,Dennis Doe,Percival Cox,2
1,John Smith,John Dorian,2
2,Random J. Patient,Elliot Reid,1
3,Dennis Doe,Molly Clock,1
4,Grace Ritchie,Elliot Reid,2
5,John Smith,Christopher Turk,1


In [32]:
query = '''
SELECT p.name AS "Patient Name",COUNT(*) as "Total Appointments"
FROM patient AS p
INNER JOIN appointment as a
    ON p.ssn = a.patient
GROUP BY p.name


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,Patient Name,Total Appointments
0,Dennis Doe,3
1,Grace Ritchie,2
2,John Smith,3
3,Random J. Patient,1


### 9.Write a query in SQL to count the number of unique patients who got an appointment for examination room C.

In [20]:

query = '''
SELECT COUNT(DISTINCT(p.name)) AS total_count_of_distinct_patient_apppinted_to_C
FROM patient AS p
INNER JOIN appointment AS a
    ON p.ssn = a.patient
WHERE examinationroom = 'C'
    

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,total_count_of_distinct_patient_apppinted_to_c
0,3


### 10.Write a query in SQL to find the name of the nurses and the room scheduled, where they will assist the physicians.

In [21]:

query = '''
SELECT n.name as Nurse_name,s.room,p.name AS Physician_name FROM
nurse as n
INNER JOIN undergoes as u
    ON n.employeeid = u.assistingnurse
INNER JOIN stay as s
    ON u.stay = s.stayid
INNER JOIN physician as p
    ON p.employeeid = u.physician
    

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,nurse_name,room,physician_name
0,Carla Espinosa,111,Christopher Turk
1,Carla Espinosa,111,John Wen
2,Laverne Roberts,112,Christopher Turk
3,Carla Espinosa,112,John Wen
4,Paul Flowers,112,Christopher Turk


### 11.Write a query in SQL to find the name of patients and their physicians who does not require any assistance of a nurse.


In [22]:


query = '''
SELECT p.name AS Patient_name, phy.name AS Physician_name,u.assistingnurse
FROM undergoes AS u
INNER JOIN patient AS p
    ON u.patient = p.ssn
INNER JOIN physician as phy
    ON u.physician = phy.employeeid
WHERE u.assistingnurse IS NULL

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,patient_name,physician_name,assistingnurse
0,Dennis Doe,Todd Quinlan,None


### 12.Write a query in SQL to obtain the name of the patients, their block, floor, and room number where they are admitted.


In [23]:

query = '''

SELECT p.name AS patient_name,room.blockfloor,stay.room FROM patient AS p
INNER JOIN undergoes as u
    ON p.ssn = u.patient
INNER JOIN stay 
    ON u.stay = stay.stayid
INNER JOIN room
    ON stay.room = room.roomnumber

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,patient_name,blockfloor,room
0,John Smith,1,111
1,John Smith,1,111
2,Dennis Doe,1,112
3,Dennis Doe,1,112
4,John Smith,1,112
5,Dennis Doe,1,112


### 13.Write a query in SQL to find the name of the patients who took an advanced appointment, and also display their physicians and medication.

In [24]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT p.name AS "Patient", phy.name AS "Physician", m.name AS "Medication"
FROM patient p
INNER JOIN prescribes ps 
    ON ps.patient=p.ssn
INNER JOIN physician phy 
    ON ps.physician=phy.employeeid
INNER JOIN medication m 
    ON ps.medication=m.code
WHERE ps.appointment IS NOT NULL;


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,Patient,Physician,Medication
0,John Smith,John Dorian,Procrastin-X
1,Dennis Doe,Molly Clock,Thesisin


### 14.Write a query in SQL to find the name and medication for those patients who did not take any appointment. 

In [25]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT p.name AS patient_name,m.name AS medication_name,m.brand
FROM patient AS p
INNER JOIN prescribes
    ON p.ssn = prescribes.patient
INNER JOIN medication AS m
    ON prescribes.medication = m.code
WHERE appointment IS NULL


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,patient_name,medication_name,brand
0,Dennis Doe,Thesisin,Foo Labs


### 15.Write a SQL query to obtain the names of all the physicians performing a medical procedure but they are not certified to perform.

In [26]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT phy.name AS physician_name 
FROM physician AS phy
INNER JOIN undergoes AS u
    ON phy.employeeid = u.physician
INNER JOIN procedure AS p
    ON u.procedure = p.code
INNER JOIN trained_in as t
    ON u.procedure = t.treatment
WHERE u.date > certificationexpires


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,physician_name
0,Todd Quinlan


### 16.Write a query in SQL to obtain the names of all the physicians, their procedure, date when the procedure was carried out and name of the patient on which procedure have been carried out but those physicians are not certified for that procedure.

In [27]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT phy.name as physician_name,patient.name AS patient_name , phy.position AS physician_position , p.name procedure_name ,u.date AS procedure_performed,certificationexpires
FROM physician as phy
INNER JOIN undergoes AS u
    ON phy.employeeid = u.physician
INNER JOIN procedure AS p
    ON u.procedure = p.code
INNER JOIN trained_in as t
    ON u.procedure = t.treatment
INNER JOIN patient
    ON u.patient = patient.ssn
WHERE u.date > certificationexpires


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,physician_name,patient_name,physician_position,procedure_name,procedure_performed,certificationexpires
0,Todd Quinlan,Dennis Doe,Surgical Attending Physician,Obfuscated Dermogastrotomy,2008-05-09,2007-12-31


### 17.Write a query in SQL to Obtain the names of all patients who has been prescribed some medication by his/her physician who has carried out primary care and the name of that physician. 

In [28]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT p.name AS patient_name,phy.name AS physician_name,m.name AS medication_name
FROM patient AS p 
INNER JOIN prescribes as pres
    ON p.ssn = pres.patient 
INNER JOIN physician as phy
    ON pres.physician = phy.employeeid
INNER JOIN medication AS m
    ON pres.medication = m.code


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,patient_name,physician_name,medication_name
0,John Smith,John Dorian,Procrastin-X
1,Dennis Doe,Molly Clock,Thesisin
2,Dennis Doe,Molly Clock,Thesisin


### 18.Write a query in SQL to obtain the nurses and the block where they are booked for attending the patients on call.

In [29]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT n.name AS nurse_name , on_call.blockfloor , on_call.blockcode
FROM nurse as n
INNER JOIN on_call
    ON n.employeeid = on_call.nurse


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df


,nurse_name,blockfloor,blockcode
0,Carla Espinosa,1,1
1,Carla Espinosa,1,2
2,Laverne Roberts,1,3
3,Paul Flowers,1,1
4,Paul Flowers,1,2
5,Paul Flowers,1,3


### 19.Write a query in SQL to make a report which will show -

#### a.name of the patient,

#### b.name of the physician who is treating him or her,

#### b.name of the nurse who is attending him or her,

#### d.which treatment is going on to the patient,

#### e.the date of release,

#### f.in which room the patient has admitted and which floor and block the room belongs to respectively. 


In [30]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT p.name AS patient_name, phy.name as physician_name,n.name AS assisting_nurse,
stay.room , room.blockfloor,room.blockcode,procedure.name AS treatment,
stay.end_time

FROM patient as p
LEFT JOIN undergoes AS u
    ON p.ssn = u.patient

LEFT JOIN physician as phy
    ON u.physician = phy.employeeid

LEFT JOIN nurse as n
    ON u.assistingnurse = n.employeeid
LEFT JOIN procedure
    ON u.procedure = procedure.code
LEFT JOIN stay
    ON u.stay = stay.stayid
LEFT JOIN room
    ON stay.room = room.roomnumber


    


'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,patient_name,physician_name,assisting_nurse,room,blockfloor,blockcode,treatment,end_time
0,John Smith,Christopher Turk,Carla Espinosa,111.0,1.0,2.0,Reversible Pancreomyoplasty,2008-05-04
1,John Smith,John Wen,Carla Espinosa,111.0,1.0,2.0,Obtuse Pyloric Recombobulation,2008-05-04
2,Dennis Doe,Christopher Turk,Laverne Roberts,112.0,1.0,2.0,Reverse Rhinopodoplasty,2008-05-03
3,Dennis Doe,Todd Quinlan,None,112.0,1.0,2.0,Obfuscated Dermogastrotomy,2008-05-03
4,John Smith,John Wen,Carla Espinosa,112.0,1.0,2.0,Follicular Demiectomy,2008-05-03
5,Dennis Doe,Christopher Turk,Paul Flowers,112.0,1.0,2.0,Complete Walletectomy,2008-05-03
6,Grace Ritchie,None,None,NaN,NaN,NaN,None,NaT
7,Random J. Patient,None,None,NaN,NaN,NaN,None,NaT


### 20.Write a query in SQL to obtain the names of all the nurses who have ever been on call for room 122.


In [31]:
conn =connect()
cur = conn.cursor()
query = '''

SELECT n.name AS nurse_name , on_call.blockfloor , on_call.blockcode,room.roomnumber
FROM nurse as n
INNER JOIN on_call
    ON n.employeeid = on_call.nurse
INNER JOIN room
    ON (on_call.blockfloor,on_call.blockcode) = (room.blockfloor,room.blockcode)
WHERE roomnumber = 122

'''
cur.execute(query)
col = [desc[0] for desc in cur.description]  

df=pd.DataFrame(columns=col,data=cur.fetchall())
df

,nurse_name,blockfloor,blockcode,roomnumber
0,Laverne Roberts,1,3,122
1,Paul Flowers,1,3,122
